In [31]:
import pandas as pd
import json

In [32]:
nameless_rules = pd.read_csv('data\\nonlabelled_combinations.csv', header=None)
print(nameless_rules)

with open('data\\elements_ordered.json', 'r') as file:
        ordered_elements = json.load(file)

print('Ordered List of element names: {}\n#of Elements: {}'.format(ordered_elements, len(ordered_elements)))


      0   1    2    3
0     3   0    0   12
1     2   0    0   14
2     1   0    0   15
3    12  15    0   16
4    16   0    0   17
..   ..  ..  ...  ...
173  32  35  226  316
174  32  35  227  317
175  32  35  228  318
176  32  35  229  319
177  32  35  230  320

[178 rows x 4 columns]
Ordered List of element names: ['Big_Tree.jpg', 'Tree.jpg', 'Stone.jpg', 'Red_Berry.jpg', 'Blue_Berry.jpg', 'Antler.jpg', 'Small_Stone.jpg', 'Stone_Tool1.jpg', 'Bough.jpg', 'Stick.jpg', 'Bark.jpg', 'Fiber.jpg', 'Twine.jpg', 'Axe.jpg', 'Log.jpg', 'Log_W_Bark.jpg', 'Half_Log.jpg', 'Quarter_Log.jpg', 'Antler_Refined.jpg', 'Stone_Tool2.jpg', 'Stone_Tool3.jpg', 'Stone_Tool4.jpg', 'Stone_Tool5.jpg', 'Big_Bough.jpg', 'Big_Bough_Refined.jpg', 'Big_Axe.jpg', 'Small_Stick.jpg', 'Brush.jpg', 'Container.jpg', 'Red_Paint.jpg', 'Blue_Paint.jpg', 'Big_Log.jpg', 'Big_Log_W_Bark.jpg', 'S_Log_Rd.jpg', 'S_Log_Sq.jpg', 'S_Log_Tr.jpg', 'R_S_Log_Rd.jpg', 'R_S_Log_Sq.jpg', 'R_S_Log_Tr.jpg', 'B_S_Log_Rd.jpg', 'B_S_Log_Sq.jpg',

In [35]:
mapping = {i+1: s for i, s in enumerate(ordered_elements)}      

# Apply the mapping to the DataFrame
nameless_rules_replaced = nameless_rules.applymap(lambda x: mapping.get(x, x))      # Does not change all the element names.
nameless_rules_replaced.to_csv('data\\labelled_combinations.csv', index=False)